In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

"""
Application de la méthode de Galerkine aux données
"""

DataFolder="C:/Users/Julie_000/Desktop/Stage/Stage_l3"
#DataFolder="E:/Clément/Julie/Stage_L3"

# Importations des librairies

In [2]:
from mat4py import loadmat #pour charger des .mat
import matplotlib
import matplotlib.pyplot as plt # pour tracer les figures
import os # pour modifier le dossier de travail
import numpy as np 
from pathlib import Path # Pour rendre les Path compatibles entre Mac et Windows
from matplotlib import rc #Les trois prochaines lignes pour que Ã§a ressemble Ã  latex
rc('font', size=16)
rc('text', usetex=True)
import scipy as sc
import matplotlib.cm as cm #colormaps
from matplotlib.colors import Normalize #Pour l'utilisation des couleurs dans quiver
from matplotlib.ticker import MaxNLocator #pour les courbes de niveau
from scipy.ndimage import gaussian_filter #Filtrage gaussien

from scipy.linalg import pinv as scpinv
from numpy.linalg import pinv as nppinv

import fonctions_projet_divergence as fpd#pour les données

colormap2=cm.PiYG#colormap

from mpl_toolkits.mplot3d import Axes3D#plot3D

# Chargement des données
## Choix des variables

In [3]:
prof=15 # Choisi sur quelle profondeur d'eau travailler (5, 10 ou 15 mm)
m=5

## Constitution des tableaux
### Données brutes

In [4]:
os.chdir(Path(DataFolder)) # se place dans le dossier où est rangée la manip
manips=loadmat('PIVnagseul.mat') # Charge les données dans la variable manip

piv=fpd.PIV(prof,manips)#données pour la profondeur

P=fpd.nb_plan(prof)#nombre de plan dans la profondeur choisie

u,v,x,y,z=fpd.donnees2(P,piv)
print(np.shape(u),np.shape(v),np.shape(x),np.shape(y),np.shape(z))

#tableau des z:
z=np.reshape(z,(P,1))
z_2D=np.hstack((z,z))
for i in range(57):
    z_2D=np.hstack((z_2D,z))
print(np.shape(z_2D))

z_3D=np.hstack((z_2D,z_2D))
for i in range(57):
    z_3D=np.hstack((z_3D,z_2D))
z_3D=np.reshape(z_3D,(P,59,59))
print(np.shape(z_3D))

(49, 60, 60) (49, 60, 60) (49, 60, 60) (49, 60, 60) (49,)
(49, 59)
(49, 59, 59)


### Divergence

In [5]:
div_2D=np.zeros((P,59,59))
dx=np.zeros((P,59,59))
dy=np.zeros((P,59,59))

for plan in range(P):
    div_2D[plan],dx[plan],dy[plan]=fpd.divergence2D_gauss(u[plan],v[plan],x[plan],y[plan],1)
    
print(np.shape(div_2D))

(49, 59, 59)


### vz

In [6]:
vz=fpd.methode_Galerkine(u,v,x,y,z,P,m,prof)
print(np.shape(vz))

(49, 59, 59)


# Test de la divergence 3D
## Dérivation de vz

In [7]:
def deriv3D(vz,z,axis):
    """Dérive un tableau 3D y par rapport à x suivant un axe donné"""
    if vz.shape!=z.shape:
        z_good_shape=z[:-1,:,:]
    else:
        z_good_shape=z
    if axis==0:
        dvz=(vz[1:,:,:]-vz[:-1,:,:])/(z_good_shape[1:,:,:]-z_good_shape[:-1,:,:])#Donc là le tableau est de taille (P-1,n,n)
        return(dvz)

In [8]:
dvz=deriv3D(vz,z_3D,0)
print(np.shape(dvz))

(48, 59, 59)


## Remise des tableaux à la bonne shape

In [9]:
div_3D=div_2D[:-1,:,:]
print(np.shape(div_3D))

div_3d=fpd.somme(div_2D,dvz)
print(div_3D)

(48, 59, 59)
[[[ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ...  3.80007037e-02
   -9.14931954e-03 -5.83167688e-02]
  [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ...  4.17935962e-02
    4.73304448e-03 -3.31817724e-02]
  [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ...  3.46993270e-02
    4.58389527e-04 -3.41857599e-02]
  ...
  [-1.85322558e-01 -1.59133239e-01 -1.03542570e-01 ...  3.06806720e-02
    2.44753321e-02  5.87524804e-03]
  [-2.29841701e-01 -1.66741682e-01 -9.42286966e-02 ...  3.70819335e-02
    3.04081398e-02  6.59163326e-03]
  [-1.15234234e-01 -1.03015144e-01 -7.46491170e-02 ...  3.61760341e-02
    3.54085528e-02  1.37727369e-02]]

 [[ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ...  0.00000000e+00
    0.00000000e+00  0.00000000e+00]
  [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ...  0.00000000e+00
    0.00000000e+00  0.00000000e+00]
  [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ...  0.00000000e+00
    0.00000000e+00  0.00000000e+00]
  ...
  [ 5.4184

In [10]:
#print(div_3D)
print(np.allclose(div_3D,np.zeros(np.shape(div_3D)),rtol=1e-10,atol=1e-10))

False


# Point de pompage
## Indices des premiers max
### Fonction

In [11]:
def points_de_pompage(div,nmax):
    indices=np.zeros((nmax,3))
    maximums=np.zeros((nmax))
    new_div=div
    for i in range(nmax):
        indices[i] = np.unravel_index(np.argmax(new_div, axis=None), div_2D.shape)
        maximums[i]=np.max(new_div)
        new_div=np.where(new_div==maximums[i],0,new_div)
    return(maximums,indices)

### Constitution des tableaux

In [12]:
nmax=20 #Choix de nmax
maximums,indices=points_de_pompage(div_2D,nmax)
    
print(indices)
print(maximums)

[[ 0. 24. 28.]
 [ 0. 24. 27.]
 [ 0. 24. 29.]
 [ 0. 25. 28.]
 [ 3. 24. 29.]
 [ 0. 23. 28.]
 [ 0. 25. 27.]
 [ 0. 24. 30.]
 [ 3. 24. 30.]
 [ 0. 23. 27.]
 [ 0. 23. 29.]
 [ 1. 24. 29.]
 [ 1. 24. 28.]
 [ 1. 25. 28.]
 [ 3. 25. 30.]
 [ 3. 24. 28.]
 [ 0. 25. 29.]
 [ 0. 23. 30.]
 [ 2. 24. 30.]
 [ 1. 24. 30.]]
[3.30105074 3.1249595  3.08234289 3.03280917 2.96931126 2.92738134
 2.89681874 2.89437129 2.84765465 2.8181298  2.76367274 2.75852015
 2.74313682 2.71487184 2.70850224 2.70477256 2.69225874 2.65851075
 2.65646282 2.62893856]


## Maximum de la dérivée de la divergence
### Fonction

In [13]:
ddiv=deriv3D(div_3D,z_3D,0)
print(np.shape(z_3D),np.shape(div_3D))
print(np.shape(ddiv))

(49, 59, 59) (48, 59, 59)
(47, 59, 59)


### Constitution des tableaux

In [14]:
nmax=20
dmaximums,dindices=points_de_pompage(ddiv,nmax)
print(dindices)
print(dmaximums)

[[11. 32. 29.]
 [46. 27. 31.]
 [11. 31. 31.]
 [11. 32. 30.]
 [36. 26. 29.]
 [36. 26. 30.]
 [46. 26. 29.]
 [44. 31. 31.]
 [23. 31. 31.]
 [24. 27. 31.]
 [43. 26. 29.]
 [33. 31. 31.]
 [36. 27. 31.]
 [43. 27. 31.]
 [46. 26. 30.]
 [43. 26. 30.]
 [11. 32. 28.]
 [25. 31. 31.]
 [19. 31. 31.]
 [30. 31. 31.]]
[7.59079686 7.42010138 7.27333171 7.01598709 6.4937046  6.34018476
 6.20517667 5.84741796 5.8414851  5.55408986 5.26190178 5.19245567
 5.15036138 5.00348248 4.89939176 4.84528358 4.63227004 4.56508721
 4.53854061 4.50390989]
